# Project 02 - Image Processing

## Thông tin sinh viên

- Họ và tên: Nguyễn Thọ Tài
- MSSV: 23127255
- Lớp: 23CLC02

## Import các thư viện liên quan

In [33]:
!python -m pip install matplotlib
!python -m pip install numpy
!python -m pip install Pillow

from PIL import Image # for read, write image
import numpy as np # for matrix compute
import matplotlib.pyplot as plt # for show image
import colorsys # for convert RGB to HSL

## Helper functions

In [34]:
# Any optional parameters beyond the required ones should be defined with default values
def read_img(img_path):
	""" Read image from img_path
	returns a 2D image (numpy array)
	"""
	return np.array(Image.open(img_path))

def show_img(img_2d):
	""" Show image
	"""
	plt.imshow(img_2d)
	plt.axis('off')
	plt.show()

def save_img(img_2d, img_path: str):
	"""	Save image to img_path
	"""
	Image.fromarray(img_2d).save(img_path)

def convert_rgb_to_hsl_loop(img_2d):
    """ Convert RGB image to HSL image
    returns a 2D image (numpy array)
    """
    norm_img = np.array(img_2d) / 255.0
    height, width, _ = norm_img.shape
    hsl_array = np.zeros_like(norm_img) # since (h,s,l) shares the same space as (r, g, b)

    for i in range(height):
        for j in range(width):
            r, g, b = norm_img[i, j]
            h, l, s = colorsys.rgb_to_hls(r, g, b)
            hsl_array[i, j] = [h, s, l]

    return hsl_array

def convert_hsl_to_rgb_loop(img_2d):
    """ Convert HSL image to RGB image
    returns a 2D image (numpy array)
    """
    height, width, _ = img_2d.shape

    rgb_array = np.zeros_like(img_2d)

    for i in range(height):
        for j in range(width):
            h, s, l = img_2d[i, j]
            r, g, b = colorsys.hls_to_rgb(h, l, s)
            rgb_array[i, j] = [r, g, b]

    rgb_array = (rgb_array * 255).astype(np.uint8)
    return Image.fromarray(rgb_array)

def convert_rgb_to_hsl(img_2d):
	""" Convert RGB image to HSL image
	returns a 2D image (numpy array)
	"""
	norm_img = np.array(img_2d) / 255.0
	rgb2hsl_vec = np.vectorize(colorsys.rgb_to_hls)
	h, l, s = rgb2hsl_vec(norm_img[..., 0], norm_img[..., 1], norm_img[..., 2]) # img[..., 0] means take all previous dim but only the 0 index of the last one. (4, 4, 3) -> (4, 3)
	return np.stack([h, s, l], axis=-1)

def convert_hsl_to_rgb(img_2d):
	""" Convert HSL image to RGB image
	returns a 2D image (numpy array)
	"""
	hsl2rgb_vec = np.vectorize(colorsys.hls_to_rgb)
	r, g, b = hsl2rgb_vec(img_2d[..., 0], img_2d[..., 1], img_2d[..., 2])
	return Image.fromarray((np.stack([r, g, b], axis=-1) * 255).astype(np.uint8))

def img_change_brightness(img_2d, brightness_factor):
	pass

def img_change_contrast(img_2d, contrast_factor):
	pass

def img_flip_horizontal(img_2d):
	return img_2d[:, ::-1]

def img_flip_vertical(img_2d):
	return img_2d[::-1, :]

def img_flip_horizontal_vertical(img_2d):
	return img_2d[::-1, ::-1]

def img_crop(img_2d, start: list[int], end: list[int]):
	height, width, _ = img_2d.shape
	x1, x2 = sorted([max(0, min(start[0], width)), max(0, min(end[0], width))])
	y1, y2 = sorted([max(0, min(start[1], height)), max(0, min(end[1], height))])

	return img_2d[y1:y2, x1:x2, :]

def img_crop_quarter_center(img_2d):
	height, width, _ = img_2d.shape
	start_height, start_width = height // 4, width // 4
	return img_crop(img_2d, [start_width, start_height], [start_width * 3, start_height * 3])

def blank_mask(img_2d):
	h, w, c = img_2d.shape
	masked_image = np.zeros_like(img_2d)
	if c == 4: # if RGBA image, set the mask to full opaque (img_2d.max() -> 1.0 or 255)
		masked_image[..., 3] = img_2d.max()
	return masked_image

def img_circle_mask(img_2d):
	height, width, _ = img_2d.shape
	Y, X = np.mgrid[0:height, 0:width]
	cx, cy = width // 2, height // 2
	r = min(cx, cx)

	mask = (X - cx) ** 2 + (Y - cy) ** 2 <= r ** 2
	masked_img = blank_mask(img_2d)
	masked_img[mask] = img_2d[mask]
	return masked_img

def rotated_ellipse_mask(x, y, a, b, theta):
	cos_t, sin_t = np.cos(theta), np.sin(theta)
	term1 = ((x * cos_t + y * sin_t) ** 2) / a ** 2
	term2 = ((x * sin_t - y * cos_t) ** 2) / b ** 2
	return (term1 + term2) <= 1 # x^2 / a^2 + y^2 / b^2 <= 1, take the inner part

def img_2ellipse_mask(img_2d: np.ndarray):
	height, width, _ = img_2d.shape
	Y, X = np.ogrid[:height, :width]
	diag = (width * width + height * height) ** 0.5 - 5

	a, b = 0.875 * diag / 2, 0.5 * diag / 2
	theta = np.deg2rad(45)

	# translate the grid coord to the center
	x = X - width // 2
	y = Y - height // 2

	frame_mask = np.logical_or(rotated_ellipse_mask(x, y, a, b, theta),
							   rotated_ellipse_mask(x, y, a, b, -theta))

	masked_img = blank_mask(img_2d)
	masked_img[frame_mask] = img_2d[frame_mask]
	return masked_img

def gen_box_blur(size: int) -> np.ndarray:
	return np.ones((size, size)) / size ** 2

MAT_BOX_BLUR_3 = gen_box_blur(3)
MAT_BOX_BLUR_5 = gen_box_blur(5)
MAT_BOX_BLUR_7 = gen_box_blur(7)
MAT_BOX_BLUR_15 = gen_box_blur(15)

MAT_GAUSS_BLUR_3 = 0.0625 * np.array(
	[[1, 2, 1],
	 [2, 4, 2],
	 [1, 2, 1]])
MAT_GAUSS_BLUR_5 = 0.00390625 * np.array(
	[[1,  4,  6,  4,  1],
	 [4, 16, 24, 16, 14],
	 [6, 24, 36, 24,  6],
	 [4, 16, 24, 16,  4],
	 [1,  4,  6,  4,  1]])

def gen_gauss_blur(size: int) -> np.ndarray:
    if size % 2 == 0 or size < 1:
        raise ValueError("Size must be odd and positive")
    kernel_1d = np.array([1, 1])
    for _ in range(size - 2):
        kernel_1d = np.convolve(kernel_1d, [1, 1])
    return np.outer(kernel_1d, kernel_1d) / np.sum(kernel_1d)**2

MAT_GAUSS_BLUR_7 = gen_gauss_blur(7)
MAT_GAUSS_BLUR_15 = gen_gauss_blur(15)
MAT_GAUSS_BLUR_21 = gen_gauss_blur(21)

MAT_UNSHARP_BLUR_5 = -0.00390625 * np.array(
	[[1,  4,    6,  4,  1],
	 [4, 16,   24, 16, 14],
	 [6, 24, -476, 24,  6],
	 [4, 16,   24, 16,  4],
	 [1,  4,    6,  4,  1]])

def kernel_convolution(img_2d: np.ndarray, kernel: np.ndarray) -> np.ndarray:
	if kernel.ndim != 2:
		raise ValueError('Input kernel must be 2D')
	if img_2d.ndim != 3:
		raise ValueError('Input image must be 3D')

	norm_img = img_2d.astype(np.float16) / 255.0
	height, width, _ = norm_img.shape
	k_height, k_width = kernel.shape
	if k_height != k_width:
		raise ValueError('Input kernel matrix must be square')

	# pad image to handle boundaries
	pad_size = k_height // 2
	pad_img = np.pad(norm_img, ((pad_size, pad_size), (pad_size, pad_size), (0, 0)), mode='constant')
	# print(pad_img.size)

	patches = np.lib.stride_tricks.sliding_window_view(pad_img, window_shape=(k_height, k_height), axis=(0, 1)) # shape: (height, width, channel, k_height, k_width)
	result = np.sum(patches * kernel[None, None, None, :, :], axis=(3, 4))
	result = np.clip(result * 255, 0, 255).astype(np.uint8)
	return result


def process_image(img_2d, func=[1, 2, 3,...]):
	""" Process image with a list of functions
	func: a list of functions to apply to the image
	return processed 2D image
	"""










## Your tests

In [35]:
import os
import tracemalloc
import time
import sys

def test_flip():
	os.makedirs("flip/", exist_ok=True)
	img = read_img("demo.png")
	flip_h_img = img_flip_horizontal(img)
	flip_v_img = img_flip_vertical(img)
	flip_hv_img = img_flip_horizontal_vertical(img)
	save_img(flip_h_img, "flip/flip_horizontal_demo.png")
	save_img(flip_v_img, "flip/flip_vertical_demo.png")
	save_img(flip_hv_img, "flip/flip_horizontal_vertical_demo.png")

def test_mask():
	os.makedirs("mask/", exist_ok=True)
	img = read_img("cat.jpg")
	circle_masked_img = img_circle_mask(img)
	ellipse_masked_img = img_2ellipse_mask(img)
	save_img(circle_masked_img, "mask/mask_circle_cat.png")
	save_img(ellipse_masked_img, "mask/mask_ellipse_cat.png")

orig_stdout = sys.stdout
def file_stdout(file: str):
	sys.stdout = open(file, 'w')

def reset_stdout():
	sys.stdout.close()
	sys.stdout = orig_stdout

In [36]:
def test_kernel_convolution_raw():
	img = np.array([
		[[1, 2, 3], [4, 5, 6], [7, 8, 9]],
		[[10, 11, 12], [13, 14, 15], [16, 17, 18]],
		[[19, 20, 21], [22, 23, 24], [25, 26, 27]]
	], dtype=np.uint8)

	kernel_3x3 = np.array([[0, 0, 0], [0, 1, 0], [0, 0, 0]], dtype=np.float32)
	result_3x3 = kernel_convolution(img, kernel_3x3)
	print(f"Result: \n{result_3x3}\n")

def benchmark_kernel_convolution(name: str, img_2d: np.ndarray, kernel: np.ndarray):
	tracemalloc.start()
	start_time = time.perf_counter()
	result = kernel_convolution(img_2d, kernel)
	end_time = time.perf_counter()
	elapsed_time = end_time - start_time
	current, peak = tracemalloc.get_traced_memory()
	tracemalloc.stop()
	print(f"Kernel: {name}")
	print(f"- Execution time: {elapsed_time:.6f} seconds")
	print(f"- Current memory usage: {current / 10**6:.2f} MB")
	print(f"- Peak memory usage: {peak / 10**6:.2f} MB")
	return result

def test_kernel_convolution():
	os.makedirs("kernel/", exist_ok=True)
	img = read_img("cat.jpg")
	start_time = time.perf_counter()

	box_blur_3_img = benchmark_kernel_convolution('box_blur_3', img, MAT_BOX_BLUR_3)
	box_blur_7_img = benchmark_kernel_convolution('box_blur_7', img, MAT_BOX_BLUR_7)
	box_blur_15_img = benchmark_kernel_convolution('box_blur_15', img, MAT_BOX_BLUR_15)

	gauss_blur_3_img = benchmark_kernel_convolution('gauss_blur_3', img, MAT_GAUSS_BLUR_3)
	gauss_blur_5_img = benchmark_kernel_convolution('gauss_blur_5', img, MAT_GAUSS_BLUR_5)
	gauss_blur_7_img = benchmark_kernel_convolution('gauss_blur_7', img, MAT_GAUSS_BLUR_7)
	gauss_blur_15_img = benchmark_kernel_convolution('gauss_blur_15', img, MAT_GAUSS_BLUR_15)
	gauss_blur_21_img = benchmark_kernel_convolution('gauss_blur_21', img, MAT_GAUSS_BLUR_21)

	unsharp_blur_img = benchmark_kernel_convolution('unsharp_blur', img, MAT_UNSHARP_BLUR_5)

	end_time = time.perf_counter()
	elapsed_time = end_time - start_time
	print(f"--------------- End in: {elapsed_time:.6f} seconds")

	save_img(box_blur_3_img, "kernel/box_blur_3_cat.png")
	save_img(box_blur_7_img, "kernel/box_blur_7_cat.png")
	save_img(box_blur_15_img, "kernel/box_blur_15_cat.png")

	save_img(gauss_blur_3_img, "kernel/gauss_blur_3_cat.png")
	save_img(gauss_blur_5_img, "kernel/gauss_blur_5_cat.png")
	save_img(gauss_blur_7_img, "kernel/gauss_blur_7_cat.png")
	save_img(gauss_blur_15_img, "kernel/gauss_blur_15_cat.png")
	save_img(gauss_blur_21_img, "kernel/gauss_blur_21_cat.png")

	save_img(unsharp_blur_img, "kernel/unsharp_blur_cat.png")
file_stdout("output/bench_normal.txt")
test_kernel_convolution()
reset_stdout()

## Main FUNCTION

Tile 1

In [49]:
def tiled_kernel_convolution(img_2d: np.ndarray, kernel: np.ndarray, tile_size: int = 256) -> np.ndarray:
	if kernel.ndim != 2:
		raise ValueError('Input kernel must be 2D')
	if img_2d.ndim != 3:
		raise ValueError('Input image must be 3D')
	if kernel.shape[0] != kernel.shape[1]:
		raise ValueError('Input kernel matrix must be square')

	norm_img = img_2d.astype(np.float32) / 255.0  # Stick with float32 for accuracy
	height, width, channels = norm_img.shape
	k_height = kernel.shape[0]
	pad_size = k_height // 2
	pad_img = np.pad(norm_img, ((pad_size, pad_size), (pad_size, pad_size), (0, 0)), mode='constant')
	result = np.zeros_like(norm_img)
	pad_tile_size = tile_size + 2 * pad_size
	com_kernel = kernel[None, None, None, :, :]
	for i in range(0, height, tile_size):
		for j in range(0, width, tile_size):
			i_end = min(i + pad_tile_size, pad_img.shape[0])
			j_end = min(j + pad_tile_size, pad_img.shape[1])
			tile = pad_img[i:i_end, j:j_end]
			tile_patches = np.lib.stride_tricks.sliding_window_view(tile, window_shape=(k_height, k_height), axis=(0, 1))
			tile_result = np.sum(tile_patches * com_kernel, axis=(3, 4))
			result[i:min(i + tile_size, height), j:min(j + tile_size, width)] = \
				tile_result[:min(tile_size, height - i), :min(tile_size, width - j)]

	return np.clip(result * 255, 0, 255).astype(np.uint8)

In [50]:
def benchmark_kernel_convolution2(name: str, img_2d: np.ndarray, kernel: np.ndarray):
	tracemalloc.start()
	start_time = time.perf_counter()
	result = tiled_kernel_convolution(img_2d, kernel)
	end_time = time.perf_counter()
	elapsed_time = end_time - start_time
	current, peak = tracemalloc.get_traced_memory()
	tracemalloc.stop()
	print(f"Kernel: {name}")
	print(f"- Execution time: {elapsed_time:.6f} seconds")
	print(f"- Current memory usage: {current / 10**6:.2f} MB")
	print(f"- Peak memory usage: {peak / 10**6:.2f} MB")
	return result

def test_kernel_convolution2():
	os.makedirs("kernel2/", exist_ok=True)
	img = read_img("cat.jpg")
	start_time = time.perf_counter()

	box_blur_3_img = benchmark_kernel_convolution2('box_blur_3', img, MAT_BOX_BLUR_3)
	box_blur_7_img = benchmark_kernel_convolution2('box_blur_7', img, MAT_BOX_BLUR_7)
	box_blur_15_img = benchmark_kernel_convolution2('box_blur_15', img, MAT_BOX_BLUR_15)

	gauss_blur_3_img = benchmark_kernel_convolution2('gauss_blur_3', img, MAT_GAUSS_BLUR_3)
	gauss_blur_5_img = benchmark_kernel_convolution2('gauss_blur_5', img, MAT_GAUSS_BLUR_5)
	gauss_blur_7_img = benchmark_kernel_convolution2('gauss_blur_7', img, MAT_GAUSS_BLUR_7)
	gauss_blur_15_img = benchmark_kernel_convolution2('gauss_blur_15', img, MAT_GAUSS_BLUR_15)
	gauss_blur_21_img = benchmark_kernel_convolution2('gauss_blur_21', img, MAT_GAUSS_BLUR_21)

	unsharp_blur_img = benchmark_kernel_convolution2('unsharp_blur', img, MAT_UNSHARP_BLUR_5)

	end_time = time.perf_counter()
	elapsed_time = end_time - start_time
	print(f"--------------- End in: {elapsed_time:.6f} seconds")

	save_img(box_blur_3_img, "kernel2/box_blur_3_cat.png")
	save_img(box_blur_7_img, "kernel2/box_blur_7_cat.png")
	save_img(box_blur_15_img, "kernel2/box_blur_15_cat.png")

	save_img(gauss_blur_3_img, "kernel2/gauss_blur_3_cat.png")
	save_img(gauss_blur_5_img, "kernel2/gauss_blur_5_cat.png")
	save_img(gauss_blur_7_img, "kernel2/gauss_blur_7_cat.png")
	save_img(gauss_blur_15_img, "kernel2/gauss_blur_15_cat.png")
	save_img(gauss_blur_21_img, "kernel2/gauss_blur_21_cat.png")

	save_img(unsharp_blur_img, "kernel2/unsharp_blur_cat.png")
file_stdout("output/bench_tiled_pre_reshape_kernel.txt")
test_kernel_convolution2()
reset_stdout()

Tile 2

In [45]:
def tiled_kernel_convolution2(img_2d: np.ndarray, kernel: np.ndarray, tile_size: int = 256) -> np.ndarray:
	if kernel.ndim != 2:
		raise ValueError('Input kernel must be 2D')
	if img_2d.ndim != 3:
		raise ValueError('Input image must be 3D')
	if kernel.shape[0] != kernel.shape[1]:
		raise ValueError('Input kernel matrix must be square')

	norm_img = img_2d.astype(np.float32) / 255.0
	height, width, channels = norm_img.shape
	k_height = kernel.shape[0]
	pad_size = k_height // 2
	pad_img = np.pad(norm_img, ((pad_size, pad_size), (pad_size, pad_size), (0, 0)), mode='constant')

	tile_shape = (tile_size + 2 * pad_size, tile_size + 2 * pad_size)
	tiles = np.lib.stride_tricks.sliding_window_view(pad_img, window_shape=tile_shape, axis=(0, 1))
	tiles = tiles[::tile_size, ::tile_size].reshape(-1, tile_shape[0], tile_shape[1], channels)

	result = np.zeros_like(norm_img)

	com_kernel = kernel[None, None, None, :, :]
	for idx, tile in enumerate(tiles):
		i = (idx // (width // tile_size)) * tile_size
		j = (idx % (width // tile_size)) * tile_size
		if i >= height or j >= width:
			continue

		tile_patches = np.lib.stride_tricks.sliding_window_view(tile, window_shape=(k_height, k_height), axis=(0, 1))
		tile_result = np.sum(tile_patches * com_kernel, axis=(3, 4))

		result_i_end = min(i + tile_size, height)
		result_j_end = min(j + tile_size, width)
		tile_i_end = min(tile_size, height - i)
		tile_j_end = min(tile_size, width - j)
		result[i:result_i_end, j:result_j_end] = tile_result[:tile_i_end, :tile_j_end]

	return np.clip(result * 255, 0, 255).astype(np.uint8)

In [46]:
def benchmark_kernel_convolution3(name: str, img_2d: np.ndarray, kernel: np.ndarray):
	tracemalloc.start()
	start_time = time.perf_counter()
	result = tiled_kernel_convolution2(img_2d, kernel)
	end_time = time.perf_counter()
	elapsed_time = end_time - start_time
	current, peak = tracemalloc.get_traced_memory()
	tracemalloc.stop()
	print(f"Kernel: {name}")
	print(f"- Execution time: {elapsed_time:.6f} seconds")
	print(f"- Current memory usage: {current / 10**6:.2f} MB")
	print(f"- Peak memory usage: {peak / 10**6:.2f} MB")
	return result

def test_kernel_convolution3():
	os.makedirs("kernel2/", exist_ok=True)
	img = read_img("cat.jpg")
	start_time = time.perf_counter()

	box_blur_3_img = benchmark_kernel_convolution2('box_blur_3', img, MAT_BOX_BLUR_3)
	box_blur_7_img = benchmark_kernel_convolution2('box_blur_7', img, MAT_BOX_BLUR_7)
	box_blur_15_img = benchmark_kernel_convolution2('box_blur_15', img, MAT_BOX_BLUR_15)

	gauss_blur_3_img = benchmark_kernel_convolution2('gauss_blur_3', img, MAT_GAUSS_BLUR_3)
	gauss_blur_5_img = benchmark_kernel_convolution2('gauss_blur_5', img, MAT_GAUSS_BLUR_5)
	gauss_blur_7_img = benchmark_kernel_convolution2('gauss_blur_7', img, MAT_GAUSS_BLUR_7)
	gauss_blur_15_img = benchmark_kernel_convolution2('gauss_blur_15', img, MAT_GAUSS_BLUR_15)
	gauss_blur_21_img = benchmark_kernel_convolution2('gauss_blur_21', img, MAT_GAUSS_BLUR_21)

	unsharp_blur_img = benchmark_kernel_convolution2('unsharp_blur', img, MAT_UNSHARP_BLUR_5)

	end_time = time.perf_counter()
	elapsed_time = end_time - start_time
	print(f"--------------- End in: {elapsed_time:.6f} seconds")

	save_img(box_blur_3_img, "kernel2/box_blur_3_cat.png")
	save_img(box_blur_7_img, "kernel2/box_blur_7_cat.png")
	save_img(box_blur_15_img, "kernel2/box_blur_15_cat.png")

	save_img(gauss_blur_3_img, "kernel2/gauss_blur_3_cat.png")
	save_img(gauss_blur_5_img, "kernel2/gauss_blur_5_cat.png")
	save_img(gauss_blur_7_img, "kernel2/gauss_blur_7_cat.png")
	save_img(gauss_blur_15_img, "kernel2/gauss_blur_15_cat.png")
	save_img(gauss_blur_21_img, "kernel2/gauss_blur_21_cat.png")

	save_img(unsharp_blur_img, "kernel2/unsharp_blur_cat.png")
file_stdout("output/bench_tiled2_pre_reshape_kernel.txt")
test_kernel_convolution3()
reset_stdout()